In [1]:
import tensorflow as tf 

In [2]:
import glob
import numpy as np
import pandas as pd 

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [4]:
def dataset_extraction(path,class_number):
    x_train=[]
    y_train=[]
    all_images=glob.glob(path+'/*.jpg')
    for path in all_images :
            img=load_img(path, target_size=(150,150))
            img=img_to_array(img)
            img=img/255.0
            x_train.append(img)
            y_train.append(class_number)
    return np.array(x_train),np.array(y_train)

In [5]:
import os 
os.getcwd()

'E:\\Anaconda\\NFT'

In [6]:
from datetime import datetime 

In [7]:
start_time = datetime.now() 
trainX_building, trainY_building  = dataset_extraction("E://seg_train/seg_train/buildings/",0)
trainX_forest,trainY_forest  = dataset_extraction("E://seg_train/seg_train/forest/",1)
trainX_glacier,trainY_glacier  = dataset_extraction("E://seg_train/seg_train/glacier/",2)
trainX_mount,trainY_mount  = dataset_extraction("E://seg_train/seg_train/mountain/",3)
trainX_sea,trainY_sea  = dataset_extraction("E://seg_train/seg_train/sea/",4)
trainX_street,trainY_street  = dataset_extraction("E://seg_train/seg_train/street/",5)

print('Time taken to run this cell is  (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Time taken to run this cell is  (hh:mm:ss.ms) 0:00:31.619432


In [8]:
start_time = datetime.now() 
testX_building, testY_building  = dataset_extraction("E://seg_test/seg_test/buildings/",0)
testX_forest,testY_forest  = dataset_extraction("E://seg_test/seg_test/forest/",1)
testX_glacier,testY_glacier  = dataset_extraction("E://seg_test/seg_test/glacier/",2)
testX_mount,testY_mount  = dataset_extraction("E://seg_test/seg_test/mountain/",3)
testX_sea,testY_sea  = dataset_extraction("E://seg_test/seg_test/sea/",4)
testX_street,testY_street  =  dataset_extraction("E://seg_test/seg_test/street/",5)

print('Time taken to run this cell is  (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Time taken to run this cell is  (hh:mm:ss.ms) 0:00:08.280853


In [9]:
import matplotlib.pyplot as plt 
%matplotlib inline

In [10]:
import math 
from random import random 

In [11]:
from PIL import Image 

In [12]:
import copy 

In [13]:
trainX_building.shape , trainX_forest.shape , trainX_glacier.shape , trainX_mount.shape, trainX_sea.shape , trainX_street.shape   

((2191, 150, 150, 3),
 (2271, 150, 150, 3),
 (2404, 150, 150, 3),
 (2512, 150, 150, 3),
 (2274, 150, 150, 3),
 (2382, 150, 150, 3))

In [14]:
def conv2d(image,kernel) :
    
    #kernel.size = (3,3) ; 
    output = [[[0 for k in range(3)]  for j in range(50)] for i in range(50) ]
    for i in range(0,150,3) :
        for j in range(0,150,3) :
            for k in range(3):
                #print(image[i:i+3][j:j+3][k].shape)
                ans = 0 
                for a in range(3):
                    for b in range(3) :
                        ans += image[i+a][j+b][k]*kernel[a][b]
                #print(ans)
                output[i//3][j//3][k] =  ans 
                
                
    return output 
     

In [15]:
kernel = [[0,-1,0], [-1, 5,-1],[0,-1,0]]

In [16]:
output = conv2d(trainX_building[1],kernel)

In [17]:
output = np.array(output)
output.shape  

(50, 50, 3)

In [18]:
class Model() :
    network = [[1 for i in range(4)] for j in range(100000)]
    it =  0 

    def input(self,input) :
        self.input = input 
        
    def output(self,output) :
        self.output = output 
        
    def add(self,input_shape , prev_shape , activation_func) :
            weight = [[random() for i in range(prev_shape)] for j in range(input_shape)]
            neuron = [random() for i in range(input_shape)]
            bias = random()

            self.network[self.it][0],self.network[self.it][1],self.network[self.it][2],self.network[self.it][3] =  weight,neuron,bias,activation_func
            self.it+=1 
            
    
            
    
    

In [19]:
def active_sum(input,weight,bias) :
    activation = bias 
    for i in range(len(weight)) : 
        activation += (weight[i]*input[i]) 
        
    return activation

In [20]:
def sigmoid(input) :
    return 1.0/(1.0 + np.exp(-input))

In [21]:
def ReLU(input) :
    if(input >0) :
        return input 
    return (0.01*input)

In [22]:
def Activation(sum_active,function) :
    if(function == 'ReLU'):
        return ReLU(sum_active)
    if(function == 'sigmoid') :
        return sigmoid(sum_active)

In [23]:
def forward_prop(network,input,bias,it) :
    #print(it)
    my_input = input
    my_bias = bias 
    for i in range(it-1) :  #Each hidden layer
        for j in range(len(network[i][1])): #For each neuron 
            sum_active = active_sum(network[i][0][j],my_input,my_bias)
            activation = Activation(sum_active,network[i][3])
            #print(activation)           
            network[i][1][j] = activation
            
        my_input = network[i][1]
        my_bias = network[i][2]
        
    for j in range(len(network[it-1][1])): #For each neuron 
            sum_active = active_sum(network[it-1][0][j],my_input,my_bias)
            activation = Activation(sum_active,network[it-1][3])      
            network[it-1][1][j] = activation
                       
    return network

In [24]:
def sig_derivative(input) :
    
    return sigmoid(input)*(1.0  - sigmoid(input))

In [25]:
def rel_derivative(input) :
    if(input >0) :
        return 1 
    return (-0.01) 

In [26]:
def derivative(input,function) :
    if(function == 'ReLU') :
       ## print("ReLu")
        return rel_derivative(input)
    if(function == 'sigmoid') :
        #print("Sigmoid")
        return sig_derivative(input)

In [27]:
def back_prop(network,input,output,learning_rate,bias,it) :
    error = [[1 for i in range(1000)] for j in range(it)]
    #For last layer 
    it-=1 
    sum_det = 0 #For bias 
    for i in range(len(network[it][1])) : #For each neuron 
        
        error[it][i] = output[i] - network[it][1][i]  #Error of ith output neuron
        #print("error in ",it," layer and ",i,"th neuron is ",error[it][i])            #Printing !!!!!!!!!!
        delta = error[it][i]*derivative(network[it][1][i],network[it][3]) 
        sum_det += delta
        error[it][i] = delta 
        
        #Now adjusting all weights to this neuron 
        for j in range(len(network[it][0][i])) :
            network[it][0][i][j] += (learning_rate*delta*network[it-1][1][j])
            
            
    network[it-1][2] *= (1 + learning_rate*sum_det)
    
    it-=1 
    
    
    ## Now doing for all layers but first 
    while(it != 0) :
        sum_det = 0
        for i in range(len(network[it][1])) : #For each neuron 
            
            err = 0 #Calculating err for each neuron which is summation(error[it+1][j]*w[j][i])
            
            for j in range(len(network[it+1][1])) :
                err += (error[it+1][j]*network[it+1][0][j][i])
                
            
            error[it][i] = err 
            delta = error[it][i]*derivative(network[it][1][i],network[it][3]) 
            error[it][i] = delta
            sum_det += delta
            for j in range(len(network[it][0][i])) :
                network[it][0][i][j] += (learning_rate*delta*network[it-1][1][j])
            
            
        network[it-1][2] *= (1 + learning_rate*sum_det)

        it-=1 
        
        
    #Now for last layer . Here the only change will be that instead of network we'll take input and input bias 
    
    sum_det = 0
    for i in range(len(network[it][1])) : #For each neuron 

        err = 0 #Calculating err for each neuron which is summation(error[it+1][j]*w[j][i])

        for j in range(len(network[it+1][1])) :
            err += (error[it+1][j]*network[it+1][0][j][i])


        error[it][i] = err 
        delta = error[it][i]*derivative(network[it][1][i],network[it][3])  
        error[it][i] = delta
        sum_det += delta
        #print(len(input))
        for j in range(len(network[it][0][i])) :
            network[it][0][i][j] += (learning_rate*delta*input[j])


    bias*= (1 + learning_rate*sum_det)
    
            
            
    
    return network
        
    

In [28]:
def normalise(input) :
    mean = sum(input) / len(input) 
    variance = sum([((x - mean) ** 2) for x in input]) / len(input)
    for k in range(len(input)) :
        input[k] = (input[k] - mean)/variance
        
    return input

In [29]:
def error_function(output,actual_output) :
    #print(output , " and  ",actual_output) 
    leng = len(output) 
    answer = 0
    for i in range(leng):
        answer += (actual_output[i]*(float(math.log(output[i]))) + (1-actual_output[i])*(float(math.log(1 - output[i]))))
        
    return answer/3 

In [80]:
def train(network,output,bias,learning_rate,it,epoch) :
    error = []
    
    for i in range(epoch):
        for j in range(100) :
            start_time = datetime.now() 
            answer = 0 
            a[j] = normalise(a[j]) 
            network = forward_prop(network,a[j],bias,it)
            network = back_prop(network,a[j],output[0],learning_rate,bias,it)
            
            
            answer += error_function(network[it-1][1],output[0])
            #print("Time taken to run a is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            b[j] = normalise(b[j]) 
            network = forward_prop(network,b[j],bias,it)
            network = back_prop(network,b[j],output[1],learning_rate,bias,it)
            
            answer += error_function(network[it-1][1],output[1])
            
            #print("Time taken to run b is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            c[j] = normalise(c[j]) 
            network = forward_prop(network,c[j],bias,it)
            network = back_prop(network,c[j],output[2],learning_rate,bias,it)
            
            answer += error_function(network[it-1][1],output[2])
            #print("Time taken to run c is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            
            d[j] = normalise(d[j] ) 
            network = forward_prop(network,d[j] ,bias,it)
            network = back_prop(network,d[j] ,output[3],learning_rate,bias,it)
            
            answer += error_function(network[it-1][1],output[3])
            #print("Time taken to run d is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            
            e[j] = normalise(e[j]) 
            network = forward_prop(network,e[j],bias,it)
            network = back_prop(network,e[j],output[4],learning_rate,bias,it)
            
            answer += error_function(network[it-1][1],output[4])
            #print("Time taken to run e is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            
            f[j] = normalise(f[j]) 
            network = forward_prop(network,f[j],bias,it)
            network = back_prop(network,f[j],output[5],learning_rate,bias,it)
            
            answer += error_function(network[it-1][1],output[5])
            #print("Time taken to run e is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
            
            
            print("Epoch number ",i," done")
            print("Time taken to run e is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
                
        error.append(answer/6000)
        
    
    return network,error

In [32]:
def predict(input,network,bias,it) :
    tmp = copy.deepcopy(network) 
    input = normalise(input) 
    my_input = input
    my_bias = bias 
    for i in range(it-1) :  #Each hidden layer
        for j in range(len(network[i][1])): #For each neuron 
            sum_active = active_sum(tmp[i][0][j],my_input,my_bias)
            activation = Activation(sum_active,tmp[i][3])      
            tmp[i][1][j] = activation
            
        my_input = tmp[i][1]
        my_bias = tmp[i][2]
        
    for j in range(len(tmp[it-1][1])): #For each neuron 
            sum_active = active_sum(tmp[it-1][0][j],my_input,my_bias)
            activation = Activation(sum_active,tmp[it-1][3])         
            tmp[it-1][1][j] = activation
            
    

    max_ind = 0 
    if(tmp[it-1][1][1] > tmp[it-1][1][0]) :
        max_ind = 1 
    if(tmp[it-1][1][2] > tmp[it-1][1][max_ind]) :
        max_ind = 2 
        
        
    
    return tmp[it-1][1]

    
    

In [56]:
trainX_building.shape , trainX_forest.shape , trainX_glacier.shape , trainX_mount.shape, trainX_sea.shape , trainX_street.shape   

((1000, 150, 150, 3),
 (1000, 150, 150, 3),
 (1000, 150, 150, 3),
 (1000, 150, 150, 3),
 (1000, 150, 150, 3),
 (1000, 150, 150, 3))

In [57]:
a = [0 for i in range(1000)]
b = [0 for i in range(1000)]
c = [0 for i in range(1000)]
d = [0 for i in range(1000)]
e = [0 for i in range(1000)]
f = [0 for i in range(1000)]

In [58]:
trainX_forest = trainX_forest[:1000]

In [59]:
trainX_glacier = trainX_glacier[:1000]

In [60]:
trainX_mount = trainX_mount[:1000]

In [61]:
trainX_sea = trainX_sea[:1000]

In [62]:
trainX_street = trainX_street[:1000]

In [63]:
trainX_building = trainX_building[:1000]

In [64]:
output= [[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]]

In [66]:
out_temp = [[[0,0,0] for i in range(50)] for j in range(50)]

In [67]:
for i in range(1000) :
    start_time = datetime.now() 
    out_temp = conv2d(trainX_building[i],kernel)
    out_temp = np.array(out_temp)
    a[i] = out_temp.flatten().reshape(out_temp.shape[0]*out_temp.shape[1]*out_temp.shape[2])
    
    out_temp = conv2d(trainX_forest[i],kernel)
    out_temp = np.array(out_temp)
    b[i] = out_temp.flatten().reshape(out_temp.shape[0]*out_temp.shape[1]*out_temp.shape[2])
    
    out_temp = conv2d(trainX_glacier[i],kernel)
    out_temp = np.array(out_temp)
    c[i] = out_temp.flatten().reshape(out_temp.shape[0]*out_temp.shape[1]*out_temp.shape[2])
    
    out_temp = conv2d(trainX_mount[i],kernel)
    out_temp = np.array(out_temp)
    d[i] = out_temp.flatten().reshape(out_temp.shape[0]*out_temp.shape[1]*out_temp.shape[2])
    
    out_temp = conv2d(trainX_sea[i],kernel)
    out_temp = np.array(out_temp)
    e[i] = out_temp.flatten().reshape(out_temp.shape[0]*out_temp.shape[1]*out_temp.shape[2])
    
    out_temp = conv2d(trainX_street[i],kernel)
    out_temp = np.array(out_temp)
    f[i] = out_temp.flatten().reshape(out_temp.shape[0]*out_temp.shape[1]*out_temp.shape[2])
    
    print('Time taken to run',i,"th it is cell is  (hh:mm:ss.ms) {}".format(datetime.now() - start_time))
    

Time taken to run 0 th it is cell is  (hh:mm:ss.ms) 0:00:01.143940
Time taken to run 1 th it is cell is  (hh:mm:ss.ms) 0:00:01.142959
Time taken to run 2 th it is cell is  (hh:mm:ss.ms) 0:00:01.135946
Time taken to run 3 th it is cell is  (hh:mm:ss.ms) 0:00:01.137126
Time taken to run 4 th it is cell is  (hh:mm:ss.ms) 0:00:01.130077
Time taken to run 5 th it is cell is  (hh:mm:ss.ms) 0:00:01.131927
Time taken to run 6 th it is cell is  (hh:mm:ss.ms) 0:00:01.125037
Time taken to run 7 th it is cell is  (hh:mm:ss.ms) 0:00:01.142942
Time taken to run 8 th it is cell is  (hh:mm:ss.ms) 0:00:01.126161
Time taken to run 9 th it is cell is  (hh:mm:ss.ms) 0:00:01.232704
Time taken to run 10 th it is cell is  (hh:mm:ss.ms) 0:00:01.157984
Time taken to run 11 th it is cell is  (hh:mm:ss.ms) 0:00:01.145937
Time taken to run 12 th it is cell is  (hh:mm:ss.ms) 0:00:01.251700
Time taken to run 13 th it is cell is  (hh:mm:ss.ms) 0:00:01.137003
Time taken to run 14 th it is cell is  (hh:mm:ss.ms) 0:00:

Time taken to run 121 th it is cell is  (hh:mm:ss.ms) 0:00:01.124078
Time taken to run 122 th it is cell is  (hh:mm:ss.ms) 0:00:01.122964
Time taken to run 123 th it is cell is  (hh:mm:ss.ms) 0:00:01.260659
Time taken to run 124 th it is cell is  (hh:mm:ss.ms) 0:00:01.138040
Time taken to run 125 th it is cell is  (hh:mm:ss.ms) 0:00:01.125989
Time taken to run 126 th it is cell is  (hh:mm:ss.ms) 0:00:01.126999
Time taken to run 127 th it is cell is  (hh:mm:ss.ms) 0:00:01.130611
Time taken to run 128 th it is cell is  (hh:mm:ss.ms) 0:00:01.277917
Time taken to run 129 th it is cell is  (hh:mm:ss.ms) 0:00:01.125988
Time taken to run 130 th it is cell is  (hh:mm:ss.ms) 0:00:01.156071
Time taken to run 131 th it is cell is  (hh:mm:ss.ms) 0:00:01.131753
Time taken to run 132 th it is cell is  (hh:mm:ss.ms) 0:00:01.190299
Time taken to run 133 th it is cell is  (hh:mm:ss.ms) 0:00:01.402251
Time taken to run 134 th it is cell is  (hh:mm:ss.ms) 0:00:01.188820
Time taken to run 135 th it is cel

Time taken to run 240 th it is cell is  (hh:mm:ss.ms) 0:00:01.126080
Time taken to run 241 th it is cell is  (hh:mm:ss.ms) 0:00:01.125942
Time taken to run 242 th it is cell is  (hh:mm:ss.ms) 0:00:01.123042
Time taken to run 243 th it is cell is  (hh:mm:ss.ms) 0:00:01.124991
Time taken to run 244 th it is cell is  (hh:mm:ss.ms) 0:00:01.127074
Time taken to run 245 th it is cell is  (hh:mm:ss.ms) 0:00:01.233713
Time taken to run 246 th it is cell is  (hh:mm:ss.ms) 0:00:01.122984
Time taken to run 247 th it is cell is  (hh:mm:ss.ms) 0:00:01.120004
Time taken to run 248 th it is cell is  (hh:mm:ss.ms) 0:00:01.130977
Time taken to run 249 th it is cell is  (hh:mm:ss.ms) 0:00:01.143908
Time taken to run 250 th it is cell is  (hh:mm:ss.ms) 0:00:01.135009
Time taken to run 251 th it is cell is  (hh:mm:ss.ms) 0:00:01.124978
Time taken to run 252 th it is cell is  (hh:mm:ss.ms) 0:00:01.129946
Time taken to run 253 th it is cell is  (hh:mm:ss.ms) 0:00:01.126034
Time taken to run 254 th it is cel

Time taken to run 359 th it is cell is  (hh:mm:ss.ms) 0:00:01.123092
Time taken to run 360 th it is cell is  (hh:mm:ss.ms) 0:00:01.120156
Time taken to run 361 th it is cell is  (hh:mm:ss.ms) 0:00:01.147944
Time taken to run 362 th it is cell is  (hh:mm:ss.ms) 0:00:01.158886
Time taken to run 363 th it is cell is  (hh:mm:ss.ms) 0:00:01.128987
Time taken to run 364 th it is cell is  (hh:mm:ss.ms) 0:00:01.120995
Time taken to run 365 th it is cell is  (hh:mm:ss.ms) 0:00:01.124991
Time taken to run 366 th it is cell is  (hh:mm:ss.ms) 0:00:01.129361
Time taken to run 367 th it is cell is  (hh:mm:ss.ms) 0:00:01.151378
Time taken to run 368 th it is cell is  (hh:mm:ss.ms) 0:00:01.119055
Time taken to run 369 th it is cell is  (hh:mm:ss.ms) 0:00:01.117992
Time taken to run 370 th it is cell is  (hh:mm:ss.ms) 0:00:01.118024
Time taken to run 371 th it is cell is  (hh:mm:ss.ms) 0:00:01.127937
Time taken to run 372 th it is cell is  (hh:mm:ss.ms) 0:00:01.123129
Time taken to run 373 th it is cel

Time taken to run 478 th it is cell is  (hh:mm:ss.ms) 0:00:01.793205
Time taken to run 479 th it is cell is  (hh:mm:ss.ms) 0:00:02.439475
Time taken to run 480 th it is cell is  (hh:mm:ss.ms) 0:00:02.599048
Time taken to run 481 th it is cell is  (hh:mm:ss.ms) 0:00:01.772259
Time taken to run 482 th it is cell is  (hh:mm:ss.ms) 0:00:01.975715
Time taken to run 483 th it is cell is  (hh:mm:ss.ms) 0:00:01.995662
Time taken to run 484 th it is cell is  (hh:mm:ss.ms) 0:00:01.666542
Time taken to run 485 th it is cell is  (hh:mm:ss.ms) 0:00:01.951782
Time taken to run 486 th it is cell is  (hh:mm:ss.ms) 0:00:01.888948
Time taken to run 487 th it is cell is  (hh:mm:ss.ms) 0:00:02.064480
Time taken to run 488 th it is cell is  (hh:mm:ss.ms) 0:00:01.684494
Time taken to run 489 th it is cell is  (hh:mm:ss.ms) 0:00:01.687486
Time taken to run 490 th it is cell is  (hh:mm:ss.ms) 0:00:02.064477
Time taken to run 491 th it is cell is  (hh:mm:ss.ms) 0:00:02.037548
Time taken to run 492 th it is cel

Time taken to run 597 th it is cell is  (hh:mm:ss.ms) 0:00:01.122463
Time taken to run 598 th it is cell is  (hh:mm:ss.ms) 0:00:01.134098
Time taken to run 599 th it is cell is  (hh:mm:ss.ms) 0:00:01.135047
Time taken to run 600 th it is cell is  (hh:mm:ss.ms) 0:00:01.144924
Time taken to run 601 th it is cell is  (hh:mm:ss.ms) 0:00:01.166883
Time taken to run 602 th it is cell is  (hh:mm:ss.ms) 0:00:01.125988
Time taken to run 603 th it is cell is  (hh:mm:ss.ms) 0:00:01.240649
Time taken to run 604 th it is cell is  (hh:mm:ss.ms) 0:00:01.125026
Time taken to run 605 th it is cell is  (hh:mm:ss.ms) 0:00:01.130064
Time taken to run 606 th it is cell is  (hh:mm:ss.ms) 0:00:01.142927
Time taken to run 607 th it is cell is  (hh:mm:ss.ms) 0:00:01.125957
Time taken to run 608 th it is cell is  (hh:mm:ss.ms) 0:00:01.128015
Time taken to run 609 th it is cell is  (hh:mm:ss.ms) 0:00:01.125990
Time taken to run 610 th it is cell is  (hh:mm:ss.ms) 0:00:01.132127
Time taken to run 611 th it is cel

Time taken to run 716 th it is cell is  (hh:mm:ss.ms) 0:00:01.134093
Time taken to run 717 th it is cell is  (hh:mm:ss.ms) 0:00:01.126986
Time taken to run 718 th it is cell is  (hh:mm:ss.ms) 0:00:01.162082
Time taken to run 719 th it is cell is  (hh:mm:ss.ms) 0:00:01.200787
Time taken to run 720 th it is cell is  (hh:mm:ss.ms) 0:00:01.124493
Time taken to run 721 th it is cell is  (hh:mm:ss.ms) 0:00:01.254173
Time taken to run 722 th it is cell is  (hh:mm:ss.ms) 0:00:01.138952
Time taken to run 723 th it is cell is  (hh:mm:ss.ms) 0:00:01.125114
Time taken to run 724 th it is cell is  (hh:mm:ss.ms) 0:00:01.131972
Time taken to run 725 th it is cell is  (hh:mm:ss.ms) 0:00:01.174814
Time taken to run 726 th it is cell is  (hh:mm:ss.ms) 0:00:01.143973
Time taken to run 727 th it is cell is  (hh:mm:ss.ms) 0:00:01.164878
Time taken to run 728 th it is cell is  (hh:mm:ss.ms) 0:00:01.137965
Time taken to run 729 th it is cell is  (hh:mm:ss.ms) 0:00:01.134964
Time taken to run 730 th it is cel

Time taken to run 835 th it is cell is  (hh:mm:ss.ms) 0:00:01.127985
Time taken to run 836 th it is cell is  (hh:mm:ss.ms) 0:00:01.129963
Time taken to run 837 th it is cell is  (hh:mm:ss.ms) 0:00:01.131985
Time taken to run 838 th it is cell is  (hh:mm:ss.ms) 0:00:01.132117
Time taken to run 839 th it is cell is  (hh:mm:ss.ms) 0:00:01.153778
Time taken to run 840 th it is cell is  (hh:mm:ss.ms) 0:00:01.270636
Time taken to run 841 th it is cell is  (hh:mm:ss.ms) 0:00:01.193774
Time taken to run 842 th it is cell is  (hh:mm:ss.ms) 0:00:01.128179
Time taken to run 843 th it is cell is  (hh:mm:ss.ms) 0:00:01.145983
Time taken to run 844 th it is cell is  (hh:mm:ss.ms) 0:00:01.125999
Time taken to run 845 th it is cell is  (hh:mm:ss.ms) 0:00:01.121999
Time taken to run 846 th it is cell is  (hh:mm:ss.ms) 0:00:01.128936
Time taken to run 847 th it is cell is  (hh:mm:ss.ms) 0:00:01.118055
Time taken to run 848 th it is cell is  (hh:mm:ss.ms) 0:00:01.139951
Time taken to run 849 th it is cel

Time taken to run 954 th it is cell is  (hh:mm:ss.ms) 0:00:01.137944
Time taken to run 955 th it is cell is  (hh:mm:ss.ms) 0:00:01.128994
Time taken to run 956 th it is cell is  (hh:mm:ss.ms) 0:00:01.130964
Time taken to run 957 th it is cell is  (hh:mm:ss.ms) 0:00:01.132970
Time taken to run 958 th it is cell is  (hh:mm:ss.ms) 0:00:01.131970
Time taken to run 959 th it is cell is  (hh:mm:ss.ms) 0:00:01.132939
Time taken to run 960 th it is cell is  (hh:mm:ss.ms) 0:00:01.133014
Time taken to run 961 th it is cell is  (hh:mm:ss.ms) 0:00:01.174812
Time taken to run 962 th it is cell is  (hh:mm:ss.ms) 0:00:01.165925
Time taken to run 963 th it is cell is  (hh:mm:ss.ms) 0:00:01.173845
Time taken to run 964 th it is cell is  (hh:mm:ss.ms) 0:00:01.163853
Time taken to run 965 th it is cell is  (hh:mm:ss.ms) 0:00:01.163933
Time taken to run 966 th it is cell is  (hh:mm:ss.ms) 0:00:01.132960
Time taken to run 967 th it is cell is  (hh:mm:ss.ms) 0:00:01.129990
Time taken to run 968 th it is cel

In [68]:
a[0].shape 

(7500,)

In [69]:
a[1].shape 

(7500,)

In [81]:
model = Model()

In [82]:
start_time = datetime.now() 
model.add(500,7500,'sigmoid')
model.add(128,500,'sigmoid')
model.add(32,128,'sigmoid')
model.add(6,32,'sigmoid')
print('Time taken to run this cell is  (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Time taken to run this cell is  (hh:mm:ss.ms) 0:00:00.405878


In [83]:
512*67500 + 128*512 + 32*128 + 6*32 

34629824

In [84]:
bias = 1
epoch = 5
learning_rate = 0.1
print(model.it)

4


In [ ]:
start_time = datetime.now() 
model.network , error  = train(model.network,output,bias,learning_rate,model.it,epoch)
print('Time taken to run this cell is  (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

<ipython-input-20-a8e3aade975b>:2: RuntimeWarning: overflow encountered in exp
  return 1.0/(1.0 + np.exp(-input))


Epoch number  0  done
Time taken to run e is cell is  (hh:mm:ss.ms) 0:00:21.437375
Epoch number  0  done
Time taken to run e is cell is  (hh:mm:ss.ms) 0:00:32.077257
